# Sachsen-Anhalt

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Sachsen-Anhalt is `DEE`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pandas.errors import ParserError
import os
import shutil
from glob import glob
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict, Tuple
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import BytesIO

from camelsp import Bundesland, Station

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Sachsen-Anhalt').input_path
BASE

'/home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt'

## Parse data

We do not have a Metadata file, but one Excel file for each station. Thus we need to parse each metadata individually and collect.

In [3]:
import patoolib

# remove folder TagMittel_DGJ_20221007 to ensure that data is extracted freshly and is up to date
if os.path.exists(f"{BASE}/TagMittel_DGJ_20221007"):
    shutil.rmtree(f"{BASE}/TagMittel_DGJ_20221007")

# extract .7z archive, make sure that 7z is installed to make patoolib work
patoolib.extract_archive(f"{BASE}/TagMittel_DGJ_LSA.7z", outdir=BASE)

patool: Extracting /home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt/TagMittel_DGJ_LSA.7z ...
patool: running /usr/bin/7z x -o/home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt -- /home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt/TagMittel_DGJ_LSA.7z
patool: ... /home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt/TagMittel_DGJ_LSA.7z extracted to `/home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt'.


'/home/alexd/Projekte/CAMELS/Github/camelsp/input_data/SA_Sachsen-Anhalt'

In [4]:
files = glob(os.path.join(f"{BASE}/TagMittel_DGJ_20221007", 'LHW_*.DGJ'))
print(f"Found {len(files)} files.")

Found 252 files.


Check the Header on the first file, as we don't have any other Metadata file. We need a ZRXP parser. Couldn't find something simple quickly, so I write my own. https://prozessing.tbbm.at/zrxp/zrxp3.0_en.pdf this is the ZRXP specification. I will only implement the relevant metadata

In [5]:
# get the names from the header
# I will skip the energy market headers and remote logger headers
HEADER = dict(
    SANR='Alphanumerical station number',
    SNAME='Station name',
    SWATER='River name',
    CMW='Values per day for equidistant time series values',
    CNAME='Parameter name',
    CNR='Parameter number',
    CUNIT='Unit of the data value column',
    RINVAL='Value for missing or invalid data record',
    RTIMELVL='Time series time level',
    TZ='time zone of all time stamps in the time series block, both header and data',
    LAYOUT='specifies the column layout for the ZRXP data'
)

def extract_file(path: str) -> Tuple[Dict[str, str], pd.DataFrame]:
    # Get the header lines first
    collection = []
    headerlines = 0

    # first read the head
    with open(path, 'rb') as f:
        # go for each line
        for l in f.readlines():
            if not l.decode('latin1').startswith('#'):
                break
            else:
                # collect the header
                collection.extend([_ for _ in l.decode('latin1').replace('#', '').split('|*|') if _ not in ('', '\n', '\r\n')])
                headerlines += 1

    # create metadata container
    meta = {}

    # go for each collected header
    for co in collection:
        HEAD = [k for k in HEADER.keys() if co.startswith(k)]
        if len(HEAD) == 1:
            HEAD = HEAD[0]
            meta[HEAD] = co.replace(HEAD, '')
        elif len(HEAD) == 0:
            if 'COMMENT' in meta:
                meta['COMMENT'] += f" {co}"
            else:
                meta["COMMENT"] = co
        elif len(HEAD) > 1:
            warnings.warn(f"Can't parse header {co}")

    # now the data
    header = meta['LAYOUT'].strip('()').split(',') if 'LAYOUT' in meta else [0, 1, 2, 3, 4, 5]
    df = pd.read_csv(path, encoding='latin1', sep=' ', header=None, skiprows=headerlines, names=header, parse_dates=[0], na_values=int(meta.get('RINVAL', '-777')))

    return meta, df

extract_file(files[-1])


({'COMMENT': 'ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXPV2R2_E SOURCESYSTEMWISKI SOURCEIDf567eda8-1628-42ec-89b9-fac8a7c8b760 TSPATH/LHW/441201/W.DGJ/Day.Mean.DGJ',
  'SANR': '441201',
  'SNAME': 'Hödingen',
  'SWATER': 'Schölecke',
  'CNR': 'W.DGJ',
  'CNAME': 'W.DGJ',
  'TZ': 'UTC+1',
  'RINVAL': '-777',
  'CUNIT': 'cm',
  'LAYOUT': '(timestamp,value,status,interpolation_type,remark)'},
        timestamp  value  status  interpolation_type remark
 0     1980-11-01    5.0      40                 603    NaN
 1     1980-11-02    5.0      40                 603    NaN
 2     1980-11-03    5.0      40                 603    NaN
 3     1980-11-04    5.0      40                 603    NaN
 4     1980-11-05   10.0      40                 603    NaN
 ...          ...    ...     ...                 ...    ...
 15062 2022-01-27   11.0      40                 603    NaN
 15063 2022-01-28   12.0      40                 603    NaN
 15064 2022-01-29   12.0      40                 603    NaN
 150

Go go for all files

In [6]:
# container
meta = []
raw_data = []

with warnings.catch_warnings(record=True) as warn:
    for fname in tqdm(files):
        m, df = extract_file(fname)
        meta.append(m)
        raw_data.append(df)

print(f"Parsed {len(meta)} files with {len(warn)} warnings.")

100%|██████████| 252/252 [00:15<00:00, 16.57it/s]

Parsed 252 files with 0 warnings.


# create metadata

This should be pretty straightforward, but maybe not super-helpful

In [7]:
metadata = pd.DataFrame(meta)
metadata

,COMMENT,SANR,SNAME,SWATER,CNR,CNAME,TZ,RINVAL,CUNIT,LAYOUT
0,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,578350,Unterrißdorf,Böse Sieben,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
1,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,594137,Karritz (Secantsgraben),Schaugraben/Secantsgraben,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
2,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,576900,Oberthau,Weiße Elster,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
3,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,579785,Derenburg (Hellbach),Hellbach,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
4,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,592002,Demker,Tanger,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
...,...,...,...,...,...,...,...,...,...,...
247,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,570340,Saaleck,Saale,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
248,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,587620,Tucheim,Tucheim-Parchener Bach,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem..."
249,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,579405,Trautenstein,Rappbode,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."
250,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,594180,Goldbeck,Uchte,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem..."


There is more metadata in the provided Pegel shapefile, including critical information such as location.

In [8]:
# read geopackage
gdf_meta = gpd.read_file(os.path.join(BASE, '../Shapes/Sachsen-Anhalt_Shapes/Datensatz.gpkg'))

# drop column geometry
gdf_meta.drop(columns='geometry', inplace=True)

# # provider_id as int
gdf_meta['PKZ'] = gdf_meta['PKZ'].astype(int)
metadata['SANR'] = metadata['SANR'].astype(int)

# # concat metadata and gdf_meta
metadata = metadata.merge(gdf_meta, left_on='SANR', right_on='PKZ', how='left')

# make Pegelkennziffer str again
metadata['SANR'] = metadata['SANR'].astype(str)

,COMMENT,SANR,SNAME,SWATER,CNR,CNAME,TZ,RINVAL,CUNIT,LAYOUT,PKZ,Pegelname,AREA_KM2,WEG_3ST,WEG_4ST,EZG außerhalb LSA
0,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,578350,Unterrißdorf,Böse Sieben,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem...",578350,Unterrißdorf,102.0,567,5672,nein
1,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,594137,Karritz (Secantsgraben),Schaugraben/Secantsgraben,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem...",594137,Karritz,187.0,591,5916,nein
2,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,576900,Oberthau,Weiße Elster,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem...",576900,Oberthau,4979.0,566,5668,"ja, komplett aber unsicher"
3,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,579785,Derenburg (Hellbach),Hellbach,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem...",579785,Derenburg,27.4,568,5686,nein
4,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,592002,Demker,Tanger,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",592002,Demker,350.0,578,5784,nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,570340,Saaleck,Saale,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",570340,Saaleck,5026.0,563,5637,"ja, komplett aber unsicher"
248,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,587620,Tucheim,Tucheim-Parchener Bach,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",587620,Tucheim,133.0,587,5874,"ja, komplett vorliegend"
249,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,579405,Trautenstein,Rappbode,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem...",579405,Trautenstein,39.2,568,5683,"ja, komplett vorliegend"
250,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,594180,Goldbeck,Uchte,W.DGJ,W.DGJ,UTC+1,-777,cm,"(timestamp,value,status,interpolation_type,rem...",594180,Goldbeck,422.0,591,5916,nein


In [9]:
id_column = 'SANR'

Metadata is seperated for Q and W data, this would result in two seperate entries in the nuts_mapping for each station:

In [10]:
print(f"Two entries for ID '578410':\n{metadata[metadata[id_column] == '578410']}")

# all IDs have two entries -> Q and W
print(f"\nNumber of duplicated IDs: {metadata[id_column].duplicated().sum()}")


Two entries for ID '578410':
                                               COMMENT    SANR   SNAME  \
7    ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...  578410  Wippra   
132  ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...  578410  Wippra   

     SWATER    CNR  CNAME     TZ RINVAL CUNIT  \
7    Wipper  Q.DGJ  Q.DGJ  UTC+1   -777  m³/s   
132  Wipper  W.DGJ  W.DGJ  UTC+1   -777    cm   

                                                LAYOUT     PKZ Pegelname  \
7    (timestamp,value,status,interpolation_type,rem...  578410    Wippra   
132  (timestamp,value,status,interpolation_type,rem...  578410    Wippra   

     AREA_KM2 WEG_3ST WEG_4ST EZG außerhalb LSA  
7       136.0     567    5676              nein  
132     136.0     567    5676              nein  

Number of duplicated IDs: 126


We have to merge the Q and W metadata first.

In [11]:
# the following columns are Q/W specific and get a Q_ or W_ prefix
update_keys = ['COMMENT', 'CNR', 'CNAME', 'CUNIT']

# get all unique ids
all_ids = metadata[id_column].unique()

# container
qw_metadata = []

# go for each id
for id in all_ids:
    # get Q metadata as dict
    q_meta = metadata[(metadata[id_column] == id) & (metadata['CNAME'] == 'Q.DGJ')].to_dict(orient='records')[0]
    # get W metadata as dict
    w_meta = metadata[(metadata[id_column] == id) & (metadata['CNAME'] == 'W.DGJ')].to_dict(orient='records')[0]

    q_updated_meta = {}
    w_updated_meta = {}

    # add prefix to Q keys
    for key in q_meta.keys():
        if key in update_keys:
            q_updated_meta[f"Q_{key}"] = q_meta[key]
        else:
            q_updated_meta[key] = q_meta[key]
    
    # add prefix to W keys
    for key in w_meta.keys():
        if key in update_keys:
            w_updated_meta[f"W_{key}"] = w_meta[key]
        else:
            w_updated_meta[key] = w_meta[key]

    # merge Q and W metadata
    
    qw_metadata.append({**q_updated_meta, **w_updated_meta})

qw_metadata = pd.DataFrame(qw_metadata)
qw_metadata
    

,Q_COMMENT,SANR,SNAME,SWATER,Q_CNR,Q_CNAME,TZ,RINVAL,Q_CUNIT,LAYOUT,PKZ,Pegelname,AREA_KM2,WEG_3ST,WEG_4ST,EZG außerhalb LSA,W_COMMENT,W_CNR,W_CNAME,W_CUNIT
0,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,578350,Unterrißdorf,Böse Sieben,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",578350,Unterrißdorf,102.0,567,5672,nein,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
1,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,594137,Karritz (Secantsgraben),Schaugraben/Secantsgraben,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",594137,Karritz,187.0,591,5916,nein,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
2,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,576900,Oberthau,Weiße Elster,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",576900,Oberthau,4979.0,566,5668,"ja, komplett aber unsicher",ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
3,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,579785,Derenburg (Hellbach),Hellbach,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",579785,Derenburg,27.4,568,5686,nein,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
4,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,592002,Demker,Tanger,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",592002,Demker,350.0,578,5784,nein,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,561150,Wolfen,Fuhne (östlich),Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",561150,Wolfen,58.9,549,5497,nein,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
122,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,591070,Wolmirstedt,Ohre,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",591070,Wolmirstedt,1504.0,576,5761,"ja, komplett vorliegend",ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
123,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,597105,Hagen,Purnitz,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",597105,Hagen,116.0,593,5934,nein,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm
124,ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,5898807,Waldfrieden,Neue Jäglitz,Q.DGJ,Q.DGJ,UTC+1,-777,m³/s,"(timestamp,value,status,interpolation_type,rem...",5898807,Waldfrieden,57.8,589,5894,"ja, unvollständig vorliegend",ZRXPVERSION2300.100 ZRXPCREATORKiIOSystem.ZRXP...,W.DGJ,W.DGJ,cm


Get all status

In [12]:
data = []
status = []
int_type = []

for m, df in zip(meta, raw_data):
    # get status
    for s in df.status.unique():
        if s not in status:
            status.append(s)
    
    # get interpolation types
    for t in df.interpolation_type.unique():
        if t not in int_type:
            int_type.append(t)
    
    # map status to quality flag (40: True, rest: NA)
    df['flag'] = df.status.map({40: True})

    # make the df
    out = df.iloc[:, :2].copy()
    out.columns = ['date', 'q' if m['CNAME'].startswith('Q') else 'w']
    out['flag'] = df['flag']
    data.append(out)

print(f"Status:              {status}")
print(f"Interpolation types: {int_type}")

Status:              [552, 40, -2147483393, 1064, -2147482369, 3112, -2147482881, 200, -2147483608]
Interpolation types: [603, 601, 604]


### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [12]:
with Bundesland('Sachsen-Anhalt') as bl:
    # save the metadata
    bl.save_raw_metadata(qw_metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for m, df in tqdm(zip(meta, data), total=len(meta)):
        # get the provider id
        provider_id = str(m[id_column])
        bl.save_timeseries(df, provider_id,)

    # check if there were warnings (there are warnings)
    if len(warn) > 0:
        log_path = bl.save_warnings(warn)
        print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DEE10000      578350  ./DEE/DEE10000/DEE10000_data.csv
1  DEE10010      594137  ./DEE/DEE10010/DEE10010_data.csv
2  DEE10020      576900  ./DEE/DEE10020/DEE10020_data.csv
3  DEE10030      579785  ./DEE/DEE10030/DEE10030_data.csv
4  DEE10040      592002  ./DEE/DEE10040/DEE10040_data.csv


  0%|          | 0/252 [00:00<?, ?it/s]

100%|██████████| 252/252 [00:05<00:00, 46.30it/s]


## Add EZG from provider to all Stations where available

In [36]:
gdf_meta = gpd.read_file(os.path.join(BASE, '../Shapes/Sachsen-Anhalt_Shapes/Datensatz.gpkg'))

for id in gdf_meta['PKZ'].values:
    # init station via PKZ, ignore warnings as we use provider_id instead of camels_id
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        s = Station(id)

    # get catchment geometry for id
    catchment = gdf_meta[gdf_meta['PKZ'] == id].iloc[[0]]

    # save catchment geometry
    s.save_catchment_geometry(catchment, datasource='federal_agency')


/home/alexd/Projekte/CAMELS/Github/camelsp/camelsp/output.py:636: UserWarning: 594143 is a provider_id and not a CAMELS-DE NUTSID. provider_id might have duplicates, using the first one: 594143
  warnings.warn(f"{provider_id} is a provider_id and not a CAMELS-DE NUTSID. provider_id might have duplicates, using the first one: {camels_id}")


'/home/alexd/Projekte/CAMELS/Github/camelsp/output_data/DEE/DEE10450/DEE10450_federal_agency_catchment.geojson'